In [15]:
class Grid:
    def __init__(self):
        self.connections = []
        
        self.max_x = 0
        self.max_y = 0
        self.min_x = 0
        self.min_y = 0
        
    def add_connection(self, x1, y1, x2, y2):
        for x in (x1, x2):
            if x > self.max_x:
                self.max_x = x
            elif x < self.min_x:
                self.min_x = x
                
        for y in (y1, y2):
            if y > self.max_y:
                self.max_y = y
            elif y < self.min_y:
                self.min_y = y
        
        self.connections.append(((x1, y1, x2, y2)))
        
    def get_normalized_connections(self):
        return [(x1 - self.min_x, y1 - self.min_y, x2 - self.min_x, y2 - self.min_y) 
                for x1, y1, x2, y2 in self.connections]
    
    def get_demensions(self):
        return self.max_x - self.min_x, self.max_y - self.min_y
    
    
grid = Grid()
grid.add_connection(-2, -1, 10, 0)
grid.add_connection(2, 5, 5, -1)
grid.min_x
print(grid.get_demensions())
grid.get_normalized_connections()

(12, 6)


[(0, 0, 12, 1), (4, 6, 7, 0)]

In [47]:
def get_svg(filename, grid, spacing=10, line_width=1, 
            line_color=(0, 0, 0), bg_color=(255, 255, 255),
            margin=None):
    
    if margin is None:
        margin = spacing
        
    connections = grid.get_normalized_connections()
    grid_width, grid_height = grid.get_demensions()
    
    width = grid_width * spacing + 2 * margin
    height = grid_height * spacing + 2 * margin
    
    svg_lines = [
        f'<svg width="{width}" height="{height}">\n'
        f'<rect x="0" y="0" width="{width}" height="{height}" fill="rgb{bg_color}"/>'
    ]
    
    for rel_x1, rel_y1, rel_x2, rel_y2 in connections:
        x1 = margin + spacing * rel_x1
        y1 = margin + spacing * rel_y1
        x2 = margin + spacing * rel_x2
        y2 = margin + spacing * rel_y2
        
        svg_lines.append(f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" '
                         f'stroke="rgb{line_color}" stroke-width="{line_width}" stroke-linecap="square"/>')
        
    svg_lines.append('</svg>')
        
    with open(filename, 'w') as file:
        file.write(''.join(svg_lines))
        
    
get_svg('svgs/grid_test', grid)

In [50]:
def get_connections_mirrored_x(connections, x_len):
    max_x = 2 * x_len - 1
    return [(max_x - x1, y1, max_x - x2, y2) for x1, y1, x2, y2 in connections]


def get_connections_mirrored_y(connections, y_len):
    max_y = 2 * y_len - 1
    return [(x1, max_y - y1, x2, max_y - y2) for x1, y1, x2, y2 in connections]


def get_pattern(n):
    mirror_along_x = True
    x_len = 1
    y_len = 1
    connections = []
    for _ in range(n):
        if mirror_along_x:
            connections.extend(get_connections_mirrored_x(connections, x_len))
            
            x1 = 0
            y1 = y_len - 1
            x2 = 0
            y2 = y_len
            x3 = 2 * x_len - 1
            y3 = y_len
            x4 = x3
            y4 = y_len - 1
            
            x_len *= 2
            y_len += 1
            
            mirror_along_x = False
        else:
            connections.extend(get_connections_mirrored_y(connections, y_len))
            
            x1 - x_len - 1
            y1 = 0
            x2 = x_len
            y2 = 0
            x3 = x_len
            y3 = 2 * y_len - 1
            x4 = x_len - 1
            y4 = y3
            
            x_len += 1
            y_len *= 2
            
            mirror_along_x = True
        
        for x1, y1, x2, y2 in ((x1, y1, x2, y2), (x2, y2, x3, y3), (x3, y3, x4, y4)):
            connections.append((x1, y1, x2, y2))
            
    grid = Grid()
    for (x1, y1, x2, y2) in connections:
        grid.add_connection(x1, y1, x2, y2)
        
    return grid


grid = get_pattern(13)
get_svg('svgs/pattern_test.svg', grid, line_width=1, spacing=2)